# Data Preparation 

# Filtrage 

In [ ]:
file=r'C:\Users\Dell\Desktop\file.edf'

In [ ]:
pip install -U mne

     |████████████████████████████████| 6.9MB 4.0MB/s 


In [ ]:
import mne
  import scipy
import matplotlib.pyplot as plt

IndentationError: ignored

In [ ]:
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()

In [ ]:
info = data.info
channels = data.ch_names

In [ ]:
raw_data

In [ ]:
import matplotlib.pyplot as plt

plt.plot(raw_data[0,:4999])
plt.title("Raw EEG, electrode 0, samples 0-4999")
plt.show()

In [ ]:
def nb_channels_file(filename):
    file = filename
    data = mne.io.read_raw_edf(file)
    channels = data.ch_names
    return len(channels)

In [ ]:
import os
os.chdir(r'C:\Users\Dell\Desktop\Données_interens')
List_files=[]
for root, dirs, files in os.walk(".", topdown = False):
   for name in files:
      print(os.path.join(root, name))
      List_files.append(os.path.join(root,name))
   for name in dirs:
      print(os.path.join(root, name))

In [ ]:
List_files_edf=[]
for filename in List_files:
    if filename.endswith("edf"):
        List_files_edf.append(filename);

In [ ]:
def fichiers_patient(ID):  #retourne une liste qui contient tous les fichiers edf d'un patient donné (determiné par un ID)
    List_fichiers_patient=[]
    for filename in List_files_edf:
         if ID in filename:
                List_fichiers_patient.append(filename)
    return List_fichiers_patient
                

In [ ]:
def affiche_signaux_patient(ID):
 for file in fichiers_patient(ID):  #afficher tous les signaux du patient avec l'ID donné
    signaux_patients_file(ID,file)

In [ ]:
def signaux_patients(ID,i):      #les signaux d'un patient donnée avec un ID sur un channel i(contenus dans tous les fichiers du patient) 
    for filename in List_files_edf:
        if ID in filename:
            file = filename
            data = mne.io.read_raw_edf(file)
            raw_data = data.get_data()
            print('patient:',ID,'channel:',i)
            plt.plot(raw_data[i,:4999])  #channel i
            plt.show()

In [ ]:
def signaux_patients_file(ID,filename):   #les signaux d'un patient dans le filename donné sur tous les channels
 file = filename
 nb_channels=nb_channels_file(file)
 data = mne.io.read_raw_edf(file)
 raw_data = data.get_data()
 for  i in range(nb_channels):
    plt.plot(raw_data[i,:4999])  #channel i
    plt.show()

In [ ]:
#une fonction qui retourne un dictionnaire tel que chaque cle représente un fichier edf 
#et chaque valeur représente le nombre de channels de ce fichier
#pour chaque patient passé avec son ID en paramètre
def patient_files_channels(ID):
    Dict = {}
    keys=fichiers_patient(ID)
    Values=[]
    for file in keys:
       Values.append(nb_channels_file(file))
    Dict = {}
    for i in range(len(keys)):
       Dict[keys[i]] = Values[i]
    return Dict;

In [ ]:
def affiche_signaux_file(file):
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
        
    for i in range(n):
        sig=data.get_data()[i]
        plt.plot(sig[:4999])
        plt.title("signal channel",i)
        plt.show()
        
  

##- Butterworth filter (Low pass filter)

- first method 1 

In [ ]:
import mne
from mne.viz import plot_filter
print('Butterworth')
sfreq = 1000.
f_p = 40.
nyq = sfreq / 2.  # the Nyquist frequency is half our sample rate
freq = [0, f_p, f_p, nyq]
gain = [1, 1, 0, 0]
flim = (1., sfreq / 2.) 
iir_params = dict(order=4, ftype='butter')
filt = mne.filter.create_filter(data.get_data()[0], sfreq=1000, l_freq=None, h_freq=40,   #filtrage du signal sur l'electrode 0
                                method='iir', iir_params=iir_params,
                                verbose=True)
fig=plot_filter(filt, sfreq, freq, gain, 'Butterworth Filter', flim=flim,
                compensate=True)

plt.show()

- second method 

In [ ]:
#Design Butterworth lowpass filter
# Args:
        #cutoff (float) : the cutoff frequency of the filter.
        #fs     (float) : the sampling rate.
        #order    (int) : order of the filter
cutoff=40.
fs=1000
# calculate the Nyquist frequency
nyq = 0.5 * fs
# design filter
low = cutoff / nyq
b, a = scipy.signal.butter(3, low, btype='low', analog=False)


In [ ]:
sig=data.get_data()[0]
filtered_signal1=scipy.signal.filtfilt(b,a,sig,axis=0)

In [ ]:
plt.plot(sig[:4999])
plt.title("original signal")
plt.show()
plt.plot(filtered_signal1[:4999])
plt.title("filtered signal : Lowpass Butterworth filter ")
plt.show()

In [ ]:
affiche_signaux_patient('00000355')

In [ ]:
def filtrage_signaux_patient_channel(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            data = mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            
            cutoff=40.
            fs=1000
            nyq = 0.5 * fs
            low = cutoff / nyq
            b, a = scipy.signal.butter(3, low, btype='low', analog=False)
            filtered_signal=scipy.signal.filtfilt(b,a,sig,axis=0)
            list_fichiers_filtred.append(filtered_signal)
        
    return list_fichiers_filtred  #retourne une liste d'array 
        

In [ ]:
#tester la fonction filtrage_signaux_patient_channel

In [ ]:
signaux_patients('00000355',0)

In [ ]:
list_signaux_patient_filtered0=filtrage_signaux_patient_channel('00000355',0) #la liste des signaux filtrés de la channel0 du patient

In [ ]:
list_signaux_patient_filtered0 #chaque array représente le signal filtré d'un fichier edf sur la channel0

In [ ]:
sig0_filtered=list_signaux_patient_filtered0[0] #sig0_filtered est le signal filtré du premier fichier edf du patient sur la channel0
plt.plot(sig0_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()
#la liste list_signaux_filtered0 contient tous les signaux filtrés de la channel0 ,on affiche le premier signal de cette liste

In [ ]:
sig1_filtered=list_signaux_filtered0[1] #sig1_filtred est signal filtré du deuxième fichier edf du patient sur la channel0
plt.plot(sig1_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()
#la liste list_signaux_filtered0 contient tous les signaux filtrés de la channel0 ,on affiche le deuxième signal de cette liste

In [ ]:
def filtrage_file_channel(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    cutoff=40.
    fs=1000
    nyq = 0.5 * fs
    low = cutoff / nyq
    b, a = scipy.signal.butter(3, low, btype='low', analog=False)
    sig=data.get_data()[i]
    filtered_signal1=scipy.signal.filtfilt(b,a,sig,axis=0)
    return filtered_signal1   #retourne un array (un signal filtré)

In [ ]:
#tester la fonction filtrage_file_channel

In [ ]:
file_test=List_files_edf[0]

In [ ]:
filtered_file_channel0=filtrage_file_channel(file_test,0)

In [ ]:
filtered_file_channel0  #c'est le résultat du signal filtré de la channel0 pour le fichier "file_test"

In [ ]:
data = mne.io.read_raw_edf(file_test)
raw_data = data.get_data()
sig=data.get_data()[0]

In [ ]:
plt.plot(sig[:4999])
plt.title("original signal channel 0")
plt.show()
plt.plot(filtered_file_channel0[:4999])
plt.title("filtered file channel 0")
plt.show()

In [ ]:
def filtrage_file(file):
    List_signal=[]
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    for i in range(n):
         List_signal.append(filtrage_file_channel(file,i)) 
    return List_signal    #retourne une liste d'array / chaque array représente le résultat d'un signal filtré sur un channel    

In [ ]:
#tester la fonction filtrage_file

In [ ]:
file_test1=List_files_edf[1]

In [ ]:
data = mne.io.read_raw_edf(file_test1)
raw_data = data.get_data()

In [ ]:
file_filtred_list=filtrage_file(file_test1)

In [ ]:
file_filtred_list

In [ ]:
sig10=data.get_data()[0]
plt.plot(sig10[:4999])
plt.title("original signal channel 0")
plt.show()
sig10_filtered=file_filtred_list[0] 
plt.plot(sig10_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig11=data.get_data()[1]
plt.plot(sig11[:4999])
plt.title("original signal channel 1")
plt.show() 
sig11_filtered=file_filtred_list[1] 
plt.plot(sig11_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()




In [ ]:
def filtrage_signaux_patient(ID):
  Dict_patient=patient_files_channels(ID)
  List_edf_filtered=[]
  keys=fichiers_patient(ID)
  for file in keys:
        List_edf_filtered.append(filtrage_file(file))
        
  return List_edf_filtered  #retourne une liste de listes (chaque liste représente les signaux filtrés d'un fichier edf )

In [ ]:
#tester la fonction filtrage_signaux_patient

In [ ]:
affiche_signaux_patient('00000355')

In [ ]:
List_filtrage_signaux_patient=filtrage_signaux_patient('00000355')

In [ ]:
List_filtrage_signaux_patient #c'est une liste de listes tel que chaque liste représente un fichier filtré

In [ ]:
Liste_fichiers_patients=fichiers_patient('00000355')
Liste_fichiers_patients

In [ ]:
file2_patient=Liste_fichiers_patients[2] #le 3ème fichier du patient avec l'id '00000355'
dataI = mne.io.read_raw_edf(file2_patient)
raw_data = dataI.get_data()
file2_patient_filtred=List_filtrage_signaux_patient[2] # le 3ème fichier du patient avec l'id '00000355' filtré 

In [ ]:
#On va vérifier que le filtrage des signaux du patient est réussi en affichant les signaux de file2_patient et file2_patient_filtred

In [ ]:
sig20=dataI.get_data()[0]
plt.plot(sig20[:4999])
plt.title("original signal channel 0")
plt.show()
sig20_filtered=file2_patient_filtred[0]
plt.plot(sig20_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig21=dataI.get_data()[1]
plt.plot(sig21[:4999])
plt.title("original signal channel 1")
plt.show() 
sig21_filtered=file2_patient_filtred[1] 
plt.plot(sig21_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
def filtrage_list_files(list_files):
    List_files_filtred=[]
    for file in list_files:
        List_files_filtred.append(filtrage_file(file))
    return List_files_filtred   #retourne une liste de listes
                                    #chaque liste est une liste d'array /chaque array est le résultat d'un filtrage d'un fichier
                                    #sur un channel 
    
        

In [ ]:
#tester la fonction filtrage_list_files

In [ ]:
List_files_edf

In [ ]:
#filtrage de tous les fichiers de notre base:

List_files_edf_filtred=filtrage_list_files(List_files_edf)


In [ ]:
List_files_edf_filtred  #une liste de listes/chaque liste est une liste d'array
                        #chaque liste représente le résultat de filtrage d'un fichier edf
                        #chaque array représente le résultat de filtrage d'un fichier edf sur un channel 

In [ ]:
# vérifier que la fonction filtrage_list_files fonctionne

In [ ]:
file_test5=List_files_edf[5]   # le 5ème fichier edf dans notre base
dataI5 = mne.io.read_raw_edf(file_test5)
raw_data = dataI5.get_data()
file5_base_filtred=List_files_edf_filtred[5] # une liste d'array qui représente le résultat de filtrage du 5ème fichier 
                                                #edf de notre base


In [ ]:
sig50=dataI5.get_data()[0]
plt.plot(sig50[:4999])
plt.title("original signal channel 0")
plt.show()
sig50_filtered=file5_base_filtred[0]
plt.plot(sig50_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig51=dataI5.get_data()[1]
plt.plot(sig51[:4999])
plt.title("original signal channel 1")
plt.show() 
sig51_filtered=file5_base_filtred[1] 
plt.plot(sig51_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()


sig52=dataI5.get_data()[2]
plt.plot(sig52[:4999])
plt.title("original signal channel 2")
plt.show() 
sig52_filtered=file5_base_filtred[2] 
plt.plot(sig51_filtered[:4999])
plt.title("filtered signal channel 2")
plt.show()



##- Chebychev filter (Low pass filter) 

- first method 

In [ ]:
import mne
from mne.viz import plot_filter
sfreq = 1000.
iir_params = dict(order=8, ftype='cheby1', rp=1.)
filt = mne.filter.create_filter(data.get_data(),sfreq, l_freq=None, h_freq=f_p,
                                method='iir', iir_params=iir_params,
                                verbose=True)
plot_filter(filt, sfreq, freq, gain,
            'Chebychev-1 order=8, ripple=1 dB', flim=flim, compensate=True)

- second method

In [ ]:
N=8 #order of the filter 
rp=1. #maximum ripple allowed
fs = 1000.     # Sample rate (Hz)
fcut = 40.  # Desired filter cutoff frequency (Hz)

# Cutoff frequency relative to the Nyquist
Wn = fcut / (0.5*fs) 

In [ ]:
b,a = scipy.signal.cheby1(N,rp,Wn,'low', analog=True,output='ba')

In [ ]:
sig=data.get_data()[0]
filtered_signal2=scipy.signal.filtfilt(b,a,sig,axis=0)

In [ ]:
plt.plot(sig[:4999])
plt.title("original signal")
plt.show()
plt.plot(filtered_signal2[:4999])
plt.title("filtered signal : Chebyshev type I filter ")
plt.show()

In [ ]:
def filtrage_signaux_patient_channel_chebv(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            fichier=data = mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            
            N=8 #order of the filter 
            rp=1. #maximum ripple allowed
            fs = 1000.     # Sample rate (Hz)
            fcut = 40.  # Desired filter cutoff frequency (Hz)

            # Cutoff frequency relative to the Nyquist
            Wn = fcut / (0.5*fs) 
            
            b,a = scipy.signal.cheby1(N,rp,Wn,'low', analog=True,output='ba')
            filtered_signal=scipy.signal.filtfilt(b,a,sig,axis=0)
            list_fichiers_filtred.append(filtered_signal)
        
    return list_fichiers_filtred
        

In [ ]:
def filtrage_file_channel_chebv(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    N=8 #order of the filter 
    rp=1. #maximum ripple allowed
    fs = 1000.     # Sample rate (Hz)
    fcut = 40.  # Desired filter cutoff frequency (Hz)

     # Cutoff frequency relative to the Nyquist
    Wn = fcut / (0.5*fs) 
            
    b,a = scipy.signal.cheby1(N,rp,Wn,'low', analog=True,output='ba')
   
    sig=data.get_data()[i]
    filtered_signal1=scipy.signal.filtfilt(b,a,sig,axis=0)
    return filtered_signal1  #retourne un array qui représente le résultat du signal filtré sur un channel i 

In [ ]:
def filtrage_file_chebv(file):
    List_signal=[]
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    for i in range(n):
         List_signal.append(filtrage_file_channel_chebv(file,i))
    return List_signal       #retourne une liste d'arrays/chaque array représente le résultat d'un signal filtré sur un channel

In [ ]:
def filtrage_signaux_patient_chebv(ID):
  Dict_patient=patient_files_channels(ID)
  List_edf_filtered=[]
  keys=fichiers_patient(ID)
  for file in keys:
        List_edf_filtered.append(filtrage_file_chebv(file))
        
  return List_edf_filtered #retourne une liste de listes/chaque liste est une liste d'arrays qui représente le résultat
                           #de filtrage d'un signal edf du patient/chaque array représente le filtrage sur un channel i 

In [ ]:
def filtrage_list_files_chebv(list_files):
    List_files_filtred=[]
    for file in list_files:
        List_files_filtred.append(filtrage_file_chebv(file))
    return List_files_filtred   #retourne une liste de listes
                                    #chaque liste est une liste d'array /chaque array est le résultat d'un filtrage d'un fichier
                                    #sur un channel 
    
        

In [ ]:
#tester la fonction filtrage_file_chebv

In [ ]:
file_test5=List_files_edf[5]

In [ ]:
data5 = mne.io.read_raw_edf(file_test5)
raw_data = data.get_data()

In [ ]:
file_filtred_list_chebv=filtrage_file_chebv(file_test5)

In [ ]:
file_filtred_list_chebv

In [ ]:
sig50=data5.get_data()[0]
plt.plot(sig50[:4999])
plt.title("original signal channel 0")
plt.show()
sig50_filtered=file_filtred_list_chebv[0] 
plt.plot(sig50_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig51=data5.get_data()[1]
plt.plot(sig51[:4999])
plt.title("original signal channel 1")
plt.show() 
sig51_filtered=file_filtred_list_chebv[1] 
plt.plot(sig51_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()


In [ ]:
#tester la fonction filtrage_signaux_patients_chebv

In [ ]:
affiche_signaux_patient('00000767')

In [ ]:
List_filtrage_signaux_patient_chebv=filtrage_signaux_patient_chebv('00000767')

In [ ]:
List_filtrage_signaux_patient_chebv

In [ ]:
Liste_fichiers_patients2=fichiers_patient('00000767')
Liste_fichiers_patients2 

In [ ]:
file6_patient=Liste_fichiers_patients2[6] #le 7ème fichier du patient avec l'id '00000767'
dataI6 = mne.io.read_raw_edf(file6_patient)
raw_data = dataI6.get_data()
file6_patient_filtred=List_filtrage_signaux_patient_chebv[6] # le 7ème fichier du patient avec l'id '00000767' filtré 


In [ ]:
#On va vérifier que le filtrage des signaux du patient est réussi en affichant les signaux de file6_patient et file6_patient_filtred

In [ ]:
sig60=dataI6.get_data()[0]
plt.plot(sig60[:4999])
plt.title("original signal channel 0")
plt.show()
sig60_filtered=file6_patient_filtred[0]
plt.plot(sig60_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig61=dataI6.get_data()[1]
plt.plot(sig61[:4999])
plt.title("original signal channel 1")
plt.show() 
sig61_filtered=file6_patient_filtred[1] 
plt.plot(sig61_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()


In [ ]:
#filtrage de tous les fichiers de notre base:

In [ ]:
filtrage_list_files_chebv(List_files_edf)

##- Savitzky golay filter (smoothing filter)

In [ ]:
import numpy as np 

In [ ]:
sig3=data.get_data()[3]

In [ ]:
x=sig3
window_length=17
polyorder=2

In [ ]:
f4=scipy.signal.savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)

In [ ]:
plt.plot(sig3[:4999])
plt.title("original signal")
plt.show()
plt.plot(f4[:4999])
plt.title("filtered signal : SG filter ")
plt.show()

In [ ]:
x=sig3
window_length=51
polyorder=3

In [ ]:
f5=scipy.signal.savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)

In [ ]:
plt.plot(sig3[:4999])
plt.title("original signal")
plt.show()
plt.plot(f5[:4999])
plt.title("filtered signal : SG filter ")
plt.show()

In [ ]:
def filtrage_signaux_patient_channel_SG(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            fichier=mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            
            x=sig
            window_length=51
            polyorder=3
        
            filtered_signal=scipy.signal.savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)
            list_fichiers_filtred.append(filtered_signal)
        
    return list_fichiers_filtred

In [ ]:
def filtrage_file_channel_SG(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    sig=data.get_data()[i]
    
    x=sig
    window_length=51
    polyorder=3
    filtered_signal=scipy.signal.savgol_filter(x, window_length, polyorder, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)
    
    
    return filtered_signal

In [ ]:
def filtrage_file_SG(file):
    List_signal=[]
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    for i in range(n):
         List_signal.append(filtrage_file_channel_SG(file,i))
    return List_signal       

In [ ]:
def filtrage_signaux_patient_SG(ID):
  Dict_patient=patient_files_channels(ID)
  List_edf_filtered=[]
  keys=fichiers_patient(ID)
  for file in keys:
        List_edf_filtered.append(filtrage_file_SG(file))
        
  return List_edf_filtered

In [ ]:
def filtrage_list_files_SG(list_files):
    List_files_filtred=[]
    for file in list_files:
        List_files_filtred.append(filtrage_file_SG(file))
    return List_files_filtred
    

In [ ]:
#tester la fonction filtrage_file_SG

In [ ]:
file_test6=List_files_edf[6] 
data6 = mne.io.read_raw_edf(file_test6)
raw_data = data.get_data()


In [ ]:
file_filtred_list_SG=filtrage_file_SG(file_test6)
file_filtred_list_SG


In [ ]:
sig60=data6.get_data()[0]
plt.plot(sig60[:4999])
plt.title("original signal channel 0")
plt.show()
sig60_filtered=file_filtred_list_SG[0] 
plt.plot(sig60_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

In [ ]:
sig61=data6.get_data()[1]
plt.plot(sig61[:4999])
plt.title("original signal channel 1")
plt.show() 
sig61_filtered=file_filtred_list_SG[1] 
plt.plot(sig61_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
#tester la fonction signaux_patients

In [ ]:
affiche_signaux_patient('00009527')

In [ ]:
List_filtrage_signaux_patient_SG=filtrage_signaux_patient_SG('00009527') 

In [ ]:
List_filtrage_signaux_patient_SG

In [ ]:
Liste_fichiers_patient3=fichiers_patient('00009527')
Liste_fichiers_patient3 

In [ ]:
file7_patient=Liste_fichiers_patient3[7] #le 8ème fichier du patient avec l'id '00009527'
dataI7 = mne.io.read_raw_edf(file7_patient)
raw_data = dataI7.get_data()
file7_patient_filtred=List_filtrage_signaux_patient_SG[7] # le 8ème fichier du patient avec l'id '00009527' filtré

In [ ]:
#On va vérifier que le filtrage des signaux du patient est réussi en affichant les signaux de file7_patient et file7_patient_filtred
#sur les channels 0 et 1 

In [ ]:
sig70=dataI7.get_data()[0]
plt.plot(sig70[:4999])
plt.title("original signal channel 0")
plt.show()
sig70_filtered=file7_patient_filtred[0]
plt.plot(sig70_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig71=dataI7.get_data()[1]
plt.plot(sig71[:4999])
plt.title("original signal channel 1")
plt.show() 
sig71_filtered=file7_patient_filtred[1] 
plt.plot(sig71_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
#tester la fonction filtrage_list_files_SG

In [ ]:
List_files_edf 

In [ ]:
#filtrage de tous les fichiers de notre base:
List_files_edf_filtred_SG= filtrage_list_files_SG(List_files_edf)

In [ ]:
List_files_edf_filtred_SG

In [ ]:
# vérifier que la fonction filtrage_list_files_SG fonctionne 


In [ ]:
file_test8=List_files_edf[8]   # le 9ème fichier edf dans notre base
dataI8 = mne.io.read_raw_edf(file_test8)
raw_data = dataI5.get_data()
file8_base_filtred=List_files_edf_filtred_SG[8] # une liste d'array qui représente le résultat de filtrage du 5ème fichier 
                                                #edf de notre base


In [ ]:
sig80=dataI8.get_data()[0]
plt.plot(sig80[:4999])
plt.title("original signal channel 0")
plt.show()
sig80_filtered=file8_base_filtred[0]
plt.plot(sig80_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

In [ ]:
sig81=dataI8.get_data()[1]
plt.plot(sig81[:4999])
plt.title("original signal channel 1")
plt.show() 
sig81_filtered=file8_base_filtred[1] 
plt.plot(sig81_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
sig82=dataI8.get_data()[2]
plt.plot(sig82[:4999])
plt.title("original signal channel 2")
plt.show() 
sig82_filtered=file5_base_filtred[2] 
plt.plot(sig82_filtered[:4999])
plt.title("filtered signal channel 2")
plt.show()

##- FFT filter (smoothing filter)

In [ ]:
sigF=data.get_data()[0]

In [ ]:
import numpy as np
import scipy.fftpack
# The FFT of the signal
sig_fft = scipy.fftpack.fft(sig)
time_step = 0.02
period = 5.


# And the power (sig_fft is of complex dtype)
power = np.abs(sig_fft)**2

# The corresponding frequencies
sample_freq = scipy.fftpack.fftfreq(sig.size, d=time_step)

# Plot the FFT power
plt.figure(figsize=(6, 5))
plt.plot(sample_freq, power)
plt.xlabel('Frequency [Hz]')
plt.ylabel('plower')

# Find the peak frequency: we can focus on only the positive frequencies
pos_mask = np.where(sample_freq > 0)
freqs = sample_freq[pos_mask]
peak_freq = freqs[power[pos_mask].argmax()]

# Check that it does indeed correspond to the frequency that we generate
# the signal with
np.allclose(peak_freq, 1./period)

# An inner plot to show the peak frequency
axes = plt.axes([0.55, 0.3, 0.3, 0.5])
plt.title('Peak frequency')
plt.plot(freqs[:8], power[:8])
plt.setp(axes, yticks=[])

# scipy.signal.find_peaks_cwt can also be used for more advanced
# peak detection

In [ ]:
high_freq_fft = sig_fft.copy()
#time_vec = np.arange(0, 6625, time_step)
high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
filtered_sigF = scipy.fftpack.ifft(high_freq_fft)

In [ ]:
plt.plot(sigF[:4999])
plt.title("original signal")
plt.show()
plt.plot(filtered_sigF[:4999])
plt.title("filtered signal : FFT filter ")
plt.show()

In [ ]:
def filtrage_signaux_patient_channel_FFT(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            fichier=data = mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            list_fichiers_filtred=[]
    
            sig_fft = scipy.fftpack.fft(sig)
            time_step = 0.02
            period = 5.
            power = np.abs(sig_fft)**2
            sample_freq = scipy.fftpack.fftfreq(sig.size, d=time_step)
            pos_mask = np.where(sample_freq > 0)
            freqs = sample_freq[pos_mask]
            peak_freq = freqs[power[pos_mask].argmax()]
            np.allclose(peak_freq, 1./period)
        
        
        
        
            high_freq_fft = sig_fft.copy()
            high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
            filtered_sig = scipy.fftpack.ifft(high_freq_fft)
            list_fichiers_filtred.append(filtered_signal)
    return list_fichiers_filtred
        

In [ ]:
def filtrage_file_channel_FFT(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    sig=data.get_data()[i]
    
    sig_fft = scipy.fftpack.fft(sig)
    time_step = 0.02
    period = 5.
    power = np.abs(sig_fft)**2
    sample_freq = scipy.fftpack.fftfreq(sig.size, d=time_step)
    pos_mask = np.where(sample_freq > 0)
    freqs = sample_freq[pos_mask]
    peak_freq = freqs[power[pos_mask].argmax()]
    np.allclose(peak_freq, 1./period)
    
    high_freq_fft = sig_fft.copy()
    high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
    filtered_sig = scipy.fftpack.ifft(high_freq_fft)
    return filtered_sig  #retourne un array qui représente le résultat du signal filtré sur un channel i 

In [ ]:
def filtrage_file_FFT(file):
    List_signal=[]
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    for i in range(n):
         List_signal.append(filtrage_file_channel_FFT(file,i))
    return List_signal       #retourne une liste d'arrays/chaque array représente le résultat d'un signal filtré sur un channel

In [ ]:
def filtrage_signaux_patient_FFT(ID):
  Dict_patient=patient_files_channels(ID)
  List_edf_filtered=[]
  keys=fichiers_patient(ID)
  for file in keys:
        List_edf_filtered.append(filtrage_file_FFT(file))
        
  return List_edf_filtered #retourne une liste de listes/chaque liste est une liste d'arrays qui représente le résultat
                           #de filtrage d'un signal edf du patient/chaque array représente le filtrage sur un channel i 

In [ ]:
def filtrage_list_files_FFT(list_files):
    List_files_filtred=[]
    for file in list_files:
        List_files_filtred.append(filtrage_file_FFT(file))
    return List_files_filtred   #retourne une liste de listes
                                    #chaque liste est une liste d'array /chaque array est le résultat d'un filtrage d'un fichier
                                    #sur un channel 
    

In [ ]:
#tester la fonction filtrage_file_FFT 

In [ ]:
file_test10=List_files_edf[10] 
data10 = mne.io.read_raw_edf(file_test10)
raw_data = data.get_data()

In [ ]:
file_filtred_list_FFT=filtrage_file_FFT(file_test10)
file_filtred_list_FFT 

In [ ]:
sigF0=data10.get_data()[0]
plt.plot(sigF0[:4999])
plt.title("original signal channel 0")
plt.show()
sigF0_filtered=file_filtred_list_FFT[0] 
plt.plot(sigF0_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sigF1=data10.get_data()[1]
plt.plot(sigF1[:4999])
plt.title("original signal channel 1")
plt.show() 
sigF1_filtered=file_filtred_list_FFT[1] 
plt.plot(sigF1_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
#tester la fonction filtrage_signaux_patients_FFT

In [ ]:
affiche_signaux_patient('00009373')

In [ ]:
List_filtrage_signaux_patient_FFT=filtrage_signaux_patient_FFT('00009373')

In [ ]:
List_filtrage_signaux_patient_FFT

In [ ]:
Liste_fichiers_patients9=fichiers_patient('00009373')
Liste_fichiers_patients9 

In [ ]:
file9_patient=Liste_fichiers_patients9[9] #le 10ème fichier du patient avec l'id '00009373'
dataI9 = mne.io.read_raw_edf(file9_patient)
raw_data = dataI9.get_data()
file9_patient_filtred=List_filtrage_signaux_patient_FFT[9] # le 10ème fichier du patient avec l'id ''00009373' filtré 


In [ ]:
#On va vérifier que le filtrage des signaux du patient est réussi en affichant les signaux de file9_patient et file9_patient_filtred


In [ ]:
sig90=dataI9.get_data()[0]
plt.plot(sig90[:4999])
plt.title("original signal channel 0")
plt.show()
sig90_filtered=file9_patient_filtred[0]
plt.plot(sig90_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sig91=dataI9.get_data()[1]
plt.plot(sig91[:4999])
plt.title("original signal channel 1")
plt.show() 
sig91_filtered=file9_patient_filtred[1] 
plt.plot(sig91_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
#filtrage de tous les fichiers de notre base:
List_files_edf_filtered_FFT=filtrage_list_files_FFT(List_files_edf)


In [ ]:
List_files_edf_filtered_FFT

##- Convolution Filtering : based on average moving box 

In [ ]:
import numpy as np

In [ ]:
sigC=data.get_data()[0]

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
filetred_signalC=smooth(sigC,30)

In [ ]:
plt.plot(sigC[:4999])
plt.title("original signal")
plt.show()
plt.plot(filetred_signalC[:4999])
plt.title("filtered signal :  convolution ")
plt.show()

In [ ]:
def filtrage_signaux_patient_channel_Conv(ID,i):
    list_fichiers=fichiers_patient(ID)
    list_fichiers_filtred=[]
    for fichier in list_fichiers:
            fichier=data = mne.io.read_raw_edf(fichier)
            raw_data = data.get_data()
            sig=data.get_data()[i]
            list_fichiers_filtred=[]
            filtered_sig =smooth(sig,30)
            list_fichiers_filtred.append(filtered_signal)
    return list_fichiers_filtred

In [ ]:
def filtrage_file_channel_Conv(file,i):  #i:channel
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    sig=data.get_data()[i]
    filtered_sig = smooth(sig,30)
    return filtered_sig  #retourne un array qui représente le résultat du signal filtré sur un channel i 

In [ ]:
def filtrage_file_Conv(file):
    List_signal=[]
    n=nb_channels_file(file)
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    for i in range(n):
         List_signal.append(filtrage_file_channel_Conv(file,i))
    return List_signal       #retourne une liste d'arrays/chaque array représente le résultat d'un signal filtré sur un channel

In [ ]:
def filtrage_signaux_patient_Conv(ID):
  Dict_patient=patient_files_channels(ID)
  List_edf_filtered=[]
  keys=fichiers_patient(ID)
  for file in keys:
        List_edf_filtered.append(filtrage_file_Conv(file))
        
  return List_edf_filtered #retourne une liste de listes/chaque liste est une liste d'arrays qui représente le résultat
                           #de filtrage d'un signal edf du patient/chaque array représente le filtrage sur un channel i 

In [ ]:
def filtrage_list_files_Conv(list_files):
    List_files_filtred=[]
    for file in list_files:
        List_files_filtred.append(filtrage_file_Conv(file))
    return List_files_filtred   #retourne une liste de listes
                                    #chaque liste est une liste d'array /chaque array est le résultat d'un filtrage d'un fichier
                                    #sur un channel 
    

In [ ]:
#tester la fonction filtrage_file_Conv

In [ ]:
file_test11=List_files_edf[11] 
data11 = mne.io.read_raw_edf(file_test11)
raw_data = data11.get_data()

In [ ]:
file_filtred_list_Conv=filtrage_file_Conv(file_test11)
file_filtred_list_Conv

In [ ]:
file_filtred_list_Conv

In [ ]:
sigC0=data11.get_data()[0]
plt.plot(sigC0[:4999])
plt.title("original signal channel 0")
plt.show()
sigC0_filtered=file_filtred_list_Conv[0] 
plt.plot(sigC0_filtered[:4999])
plt.title("filtered signal channel 0")
plt.show()

sigC1=data11.get_data()[1]
plt.plot(sigC1[:4999])
plt.title("original signal channel 1")
plt.show() 
sigC1_filtered=file_filtred_list_Conv[1] 
plt.plot(sigC1_filtered[:4999])
plt.title("filtered signal channel 1")
plt.show()

In [ ]:
#filtrage de tous les fichiers de notre base:
List_files_edf_filtered_Conv=filtrage_list_files_Conv(List_files_edf) 


In [ ]:
List_files_edf_filtered_Conv

# Decomposition 

In [ ]:
import mne
import matplotlib.pyplot as plt

In [ ]:
file=r'C:\Users\Dell\Desktop\file.edf'

In [ ]:
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()

In [ ]:
channels = data.ch_names
channels

In [ ]:
import matplotlib.pyplot as plt

plt.plot(raw_data[0,:4999])
plt.title("Raw EEG, electrode 0, samples 0-4999")
plt.show()

In [ ]:
import pywt

In [ ]:
from pywt import Wavelet

wavelet = Wavelet('Haar')
print(wavelet)

In [ ]:
import matplotlib.pyplot as plt

def plot_wavelet(wavelet):

    fig, axs = plt.subplots(3,2, figsize=(11,11), squeeze =True, sharex=True, sharey=True)
    axs = axs.flatten()
    for i, level in enumerate([1, 2, 3, 4, 10, 15]):
        try:
            [psi, x] = wavelet.wavefun(level=level)
        except:
            [phi, psi, x] = wavelet.wavefun(level=level)
        axs[i].plot(x, psi)
        axs[i].text(0.85, 0.9, "Level = "+str(level), ha='center', va='center', transform=axs[i].transAxes)

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()

plot_wavelet(wavelet)

- The Daubechies 4 wavelet (db4) is the most commonly used wavelet for EEG and epilepsy seizure detection as it is smooths the frequency filtering enough to characterise the EEG, but is also computationally efficient

In [ ]:
plot_wavelet(Wavelet('db4'))

In [ ]:
from pywt import wavedec

# number of levels we are interested in
level = 6

# transpose the data because its a time-series package
data_t = raw_data[0].transpose()

# get the wavelet coefficients at each level in a list
coeffs_list = wavedec(data_t, wavelet='db4', level=level)

print(type(coeffs_list))
print('\nExample first array')
coeffs_list

In [ ]:
coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']

In [ ]:
L=[[]]
for i in range(len(coefficients)):
    array=coeffs_list[i].flatten()
    list1=array.tolist()
    L.append(list1)

In [ ]:
L

In [ ]:
len(L)

In [ ]:
L.remove(L[0])

In [ ]:
len(L)

In [ ]:
L[0]

In [ ]:
L[1]

In [ ]:
df = pd.DataFrame(columns=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1'])

In [ ]:
Series_coefficients=[]
for  i in range(len(coeffs_list)):
    Series_coefficients.append(pd.Series(L[i]))

In [ ]:
Series_coefficients

In [ ]:
len(Series_coefficients)

In [ ]:
Series_coefficients[0]

In [ ]:
Series_coefficients[1]

- Range of frequencies:

D1:  (Gamma)

D2: (Beta/Gamma)

D3: (Beta)

D4:  (Alpha)

D5:  (Theta)

D6:  (Delta)

A6:  (Delta/DC)

In [ ]:
for i in range(len(coefficients)):
   df[coefficients[i]]=Series_coefficients[i]

In [ ]:
df

In [ ]:
data1=raw_data[0]

In [ ]:
def decompose_signal_channel(file,channel):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    data1=raw_data[channel]
    channel_name=channels[channel]
    print(channel_name)
    # number of levels we are interested in
    level = 6

    # transpose the data because its a time-series package
    data_t = data1.transpose()

   # get the wavelet coefficients at each level in a list
    coeffs_list = wavedec(data1, wavelet='db4', level=level)
   
    coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
    
   
    L=[[]]
    for i in range(len(coefficients)):
      array=coeffs_list[i].flatten()
      list1=array.tolist()
      L.append(list1)
        
    L.remove(L[0])
    df = pd.DataFrame(columns=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1'])
    Series_coefficients=[]
    for  i in range(len(coeffs_list)):
     Series_coefficients.append(pd.Series(L[i]))
    
    
    for i in range(len(coefficients)):
      df[coefficients[i]]=Series_coefficients[i]
    
    return(df)

In [ ]:
df0=decompose_signal_channel(file,0)
df0

In [ ]:
df1=decompose_signal_channel(file,1)
df1

In [ ]:
pd.concat([df0,df1],axis=1)

In [ ]:
data = mne.io.read_raw_edf(file)
raw_data = data.get_data()
channels = data.ch_names
nb_channels=len(channels)

In [ ]:
nb_channels

In [ ]:
def decompose_signal_all_channels(file):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    nb_channels=len(channels)
    df_signal=decompose_signal_channel(file,0)
    for i in range(1,36):
      df_channel=decompose_signal_channel(file,i)
      df_concat=pd.concat([df_signal,df_channel],axis=1)
      df_signal=df_concat
    return df_signal

In [ ]:
decompose_signal_all_channels(file)

# Feature Extraction 

- Log Sum

In [ ]:
df

In [ ]:
df.shape

In [ ]:
List_one=[1 for i in range(5182)]

In [ ]:
List_one

In [ ]:
Serie_one=pd.Series(List_one)

In [ ]:
Serie_one

In [ ]:
def minus_small(data):    
  # find the smallest value for each data column (channel)...
  min_val = data.min()
  # ...and subtract it from all the data in the column and add one
  List_one=[1 for i in range(5182)]
  Series_one=pd.Series(List_one)
  data_substract = np.subtract(data,min_val)
  data_modified=np.add(data_substract,List_one)


  return data

In [ ]:
minus_small(df['A6'])

In [ ]:
df.sum(axis=0)

In [ ]:
df_sum=df.sum(axis=0)
df_sum

In [ ]:
absolute_sum=df.sum(axis=0)

In [ ]:
absolute_sum

In [ ]:
#extraire le minimum de chaque colonne
#pour chaque valeur fel colonne ,ena7i menha el minimum et j'ajoute un 1 
#

In [ ]:
def ave(data, output=False):
    # get the mean
    mean_data = data.mean()
    return(mean_data)
    
    

In [ ]:
ave(df['A6'])

In [ ]:
def mean_abs(data, output=False):
    # get the mean of the absolute values
    mean_abs_data = data.abs().mean()
    return mean_abs_data

In [ ]:
mean_abs(df['A6'])

In [ ]:
def coeff_std(data, output=False):
    # get the standard deviation of the coeficients
    std_data = data.std()
    return std_data

In [ ]:
coeff_std(df['A6'])

In [ ]:
#df = pd.DataFrame(columns=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1'])
#Series_coefficients=[]
#for  i in range(len(coeffs_list)):
    #Series_coefficients.append(pd.Series(L[i]))
#for i in range(len(coefficients)):
   #df[coefficients[i]]=Series_coefficients[i]
    


In [ ]:
List_average=[]
coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
len_coefficients=len(coefficients)
for i in range(len_coefficients):
    List_average.append(ave(df[coefficients[i]]))


In [ ]:
List_average

In [ ]:
coefficients_average=['Mean_A6', 'Mean_D6', 'Mean_D5', 'Mean_D4', 'Mean_D3', 'Mean_D2', 'Mean_D1']
df_average = pd.DataFrame(columns=['Mean_A6', 'Mean_D6', 'Mean_D5', 'Mean_D4', 'Mean_D3', 'Mean_D2', 'Mean_D1'])
Series_average=[]
for  i in range(len(coefficients_average)):
    Series_average.append(pd.Series(List_average[i]))
    
for i in range(len(coefficients_average)):
      df_average[coefficients_average[i]]=Series_average[i]
        
df_average
    


In [ ]:
def ave(data, output=False):
    # get the mean
    mean_data = data.mean()
    return(mean_data)

In [ ]:
def average_signal_channel(file,channel):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    data1=raw_data[channel]
   

   
    df=decompose_signal_channel(file,channel)


    List_average=[]
    coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
    len_coefficients=len(coefficients)
    for i in range(len_coefficients):
       List_average.append(ave(df[coefficients[i]]))

    
    coefficients_average=['Mean_A6', 'Mean_D6', 'Mean_D5', 'Mean_D4', 'Mean_D3', 'Mean_D2', 'Mean_D1']
    
    
    df_average = pd.DataFrame(columns=['Mean_A6', 'Mean_D6', 'Mean_D5', 'Mean_D4', 'Mean_D3', 'Mean_D2', 'Mean_D1'])
    Series_average=[]
    for  i in range(len(coefficients_average)):
        Series_average.append(pd.Series(List_average[i]))
    
    for i in range(len(coefficients_average)):
        df_average[coefficients_average[i]]=Series_average[i]
        

   
    return(df_average)

In [ ]:
average_signal_channel(file,0)

In [ ]:
def average_signal_all_channels(file):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    nb_channels=len(channels)
    df_signal=average_signal_channel(file,0)
    for i in range(1,36):
      df_channel=average_signal_channel(file,i)
      df_concat=pd.concat([df_signal,df_channel],axis=1)
      df_signal=df_concat
    return df_signal

In [ ]:
average_signal_all_channels(file)

In [ ]:
def mean_abs(data, output=False):
    # get the mean of the absolute values
    mean_abs_data = data.abs().mean()
    return mean_abs_data

In [ ]:
def mean_abs_signal_channel(file,channel):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    data1=raw_data[channel]
   

   
    df=decompose_signal_channel(file,channel)


    List_meanabs=[]
    coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
    len_coefficients=len(coefficients)
    for i in range(len_coefficients):
        List_meanabs.append(mean_abs(df[coefficients[i]]))

    
    coefficients_meanabs=['MeanAbs_A6', 'MeanAbs_D6', 'MeanAbs_D5', 'MeanAbs_D4', 'MeanAbs_D3', 'MeanAbs_D2', 'MeanAbs_D1']
    
    
    df_meanabs = pd.DataFrame(columns=['MeanAbs_A6', 'MeanAbs_D6', 'MeanAbs_D5', 'MeanAbs_D4', 'MeanAbs_D3', 'MeanAbs_D2', 'MeanAbs_D1'])
    Series_meanabs=[]
    for  i in range(len(coefficients_average)):
        Series_meanabs.append(pd.Series(List_meanabs[i]))
    
    for i in range(len(coefficients_average)):
        df_meanabs[coefficients_meanabs[i]]=Series_meanabs[i]
        

   
    return(df_meanabs)

In [ ]:
mean_abs_signal_channel(file,0)

In [ ]:
def mean_abs_signal_all_channels(file):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    nb_channels=len(channels)
    df_signal=mean_abs_signal_channel(file,0)
    for i in range(1,36):
      df_channel=mean_abs_signal_channel(file,i)
      df_concat=pd.concat([df_signal,df_channel],axis=1)
      df_signal=df_concat
    return df_signal

In [ ]:
mean_abs_signal_all_channels(file)

In [ ]:
def coeff_std(data, output=False):
    # get the standard deviation of the coeficients
    std_data = data.std()
    return std_data

In [ ]:
def coeff_std_signal_channel(file,channel):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    data1=raw_data[channel]
   

   
    df=decompose_signal_channel(file,channel)


    List_coeff_std=[]
    coefficients=['A6', 'D6', 'D5', 'D4', 'D3', 'D2', 'D1']
    len_coefficients=len(coefficients)
    for i in range(len_coefficients):
        List_coeff_std.append(coeff_std(df[coefficients[i]]))

    
    coefficients_std=['CoeffSTD_A6', 'CoeffSTD_D6', 'CoeffSTD_D5', 'CoeffSTD_D4', 'CoeffSTD_D3', 'CoeffSTD_D2', 'CoeffSTD_D1']
    
    
    df_coeffstd = pd.DataFrame(columns=['CoeffSTD_A6', 'CoeffSTD_D6', 'CoeffSTD_D5', 'CoeffSTD_D4', 'CoeffSTD_D3', 'CoeffSTD_D2', 'CoeffSTD_D1'])
    Series_coeffstd=[]
    for  i in range(len(coefficients_std)):
        Series_coeffstd.append(pd.Series(List_coeff_std[i]))
    
    for i in range(len(coefficients_average)):
        df_coeffstd[coefficients_std[i]]=Series_coeffstd[i]
        

   
    return(df_coeffstd)

In [ ]:
coeff_std_signal_channel(file,0)

In [ ]:
def coeff_std_signal_all_channels(file):
    data = mne.io.read_raw_edf(file)
    raw_data = data.get_data()
    channels = data.ch_names
    nb_channels=len(channels)
    df_signal=coeff_std_signal_channel(file,0)
    for i in range(1,36):
      df_channel=coeff_std_signal_channel(file,i)
      df_concat=pd.concat([df_signal,df_channel],axis=1)
      df_signal=df_concat
    return df_signal

In [ ]:
coeff_std_signal_all_channels(file)